# SemEval 2026 Task 8: Multi-Turn RAG Evaluation

## Task C: Retrieval-Augmented Generation

This notebook implements **Task C**: retrieving relevant documents and generating answers grounded in that context.

**Note:** Uses the centralized `src.generation` module for LLM and RAG prompts.

In [ ]:
import os
import sys
import json
from tqdm import tqdm

# Project Root
if os.path.exists("src"):
    PROJECT_ROOT = os.getcwd()
else:
    PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path: sys.path.insert(0, PROJECT_ROOT)

from src.ingestion import load_and_chunk_data, build_vector_store
from src.retrieval import get_retriever, get_qdrant_client
from src.generation import create_generation_components

In [ ]:
TEAM_NAME = "Gbgers"
DOMAINS = ["govt", "clapnq", "fiqa", "cloud"]
COLLECTION_NAME = "mtrag_unified"
TEST_MODE = True
TEST_QUERY_LIMIT = 5

CORPUS_BASE_DIR = os.path.join(PROJECT_ROOT, "dataset/corpora/passage_level")
CONVERSATIONS_FILE = os.path.join(PROJECT_ROOT, "dataset/human/conversations/conversations.json")
QDRANT_PATH = os.path.join(PROJECT_ROOT, "qdrant_db")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data/submissions")
OUTPUT_FILE = os.path.join(OUTPUT_DIR, f"submission_TaskC_{TEAM_NAME}.jsonl")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(QDRANT_PATH, exist_ok=True)

In [ ]:
# Build Index (skip if exists)
if not os.path.exists(QDRANT_PATH):
    print("Building Index...")
    all_docs = []
    for domain in DOMAINS:
        path = os.path.join(CORPUS_BASE_DIR, f"{domain}.jsonl")
        if os.path.exists(path):
            docs = load_and_chunk_data(path)
            for d in docs: d.metadata["domain"] = domain
            if TEST_MODE: docs = docs[:1000]
            all_docs.extend(docs)
    build_vector_store(all_docs, persist_dir=QDRANT_PATH, collection_name=COLLECTION_NAME)

In [ ]:
print("Initializing Components...")
retriever = get_retriever(qdrant_path=QDRANT_PATH, collection_name=COLLECTION_NAME)
# uses default cached model: meta-llama/Llama-3.1-8B-Instruct
components = create_generation_components()

In [ ]:
def extract_last_query(messages):
    for msg in reversed(messages):
        if msg.get("speaker") == "user": return msg.get("text", "")
    return ""

with open(CONVERSATIONS_FILE) as f: all_convs = json.load(f)

results = []
for domain in DOMAINS:
    convs = [c for c in all_convs if domain in c.get("domain", "").lower()]
    if TEST_MODE: convs = convs[:TEST_QUERY_LIMIT]
    
    for conv in tqdm(convs, desc=domain):
        q = extract_last_query(conv.get("messages", []))
        if not q: continue
        
        # 1. Retrieve
        docs = retriever.invoke(q)
        
        # 2. Format Context
        context_text = ""
        contexts = []
        for i, d in enumerate(docs):
            txt = d.metadata.get("parent_text") or d.page_content
            context_text += f"[Document {i+1}]\n{txt}\n\n"
            contexts.append({
                "document_id": str(d.metadata.get("doc_id") or f"{domain}_{i}"),
                "score": float(d.metadata.get("relevance_score") or 0.0),
                "text": txt
            })
        
        # 3. Generate with RAG (using PRE-BUILT generator from src.generation)
        try:
            # src.generation.generator expects 'context' and 'question'
            ans = components.generator.invoke({"context": context_text, "question": q})
        except Exception as e:
            ans = str(e)
        
        results.append({
            "conversation_id": conv.get("author"),
            "task_id": f"{conv.get('author')}::1",
            "Collection": f"mt-rag-{domain}",
            "contexts": contexts,
            "predictions": [{"text": ans}]
        })

with open(OUTPUT_FILE, 'w') as f:
    for r in results: f.write(json.dumps(r) + '\n')
print("Saved.")